# Tutorial: Variational algorithms on Hybrid jobs

## Hybrid Jobs

Amazon Braket Hybrid Jobs, a capability of Amazon Braket that simplifies the process of setting up and executing quantum algorithms on quantum cpus with classical computing capability like preprocessing data, post-processing the results, logging, and scheduling.

## Prepare Files

### Prepare Hybrid Job Script

In [ ]:
! cat ./data/2_hybrid_jobs/job_script.py

"""Example of Hybrid Job payload with VQE."""
from braket.jobs import save_job_result
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator

from qiskit_braket_provider import AWSBraketProvider


def main():
    backend = AWSBraketProvider().get_backend("SV1")

    h2_op = SparsePauliOp(
        ["II", "IZ", "ZI", "ZZ", "XX"],
        coeffs=[
            -1.052373245772859,
            0.39793742484318045,
            -0.39793742484318045,
            -0.01128010425623538,
            0.18093119978423156,
        ],
    )

    estimator = BackendEstimator(
        backend=backend,
        options={"seed_simulator": 42, "seed_transpiler": 42, "shots": 10},
        skip_transpilation=False,
    )
    ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
    slsqp = SLSQP(maxiter=1)

    vqe = 

## Running your job
The Braket-managed base container have `qiskit-braket-provider` preinstalled and ready to use for running your variational algorithms. To use the base container, call retrieve_image(Framework.BASE, AwsSession().region) as demonstrated below:

In [ ]:
from pprint import pprint
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.image_uris import Framework, retrieve_image

job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="data/2_hybrid_jobs/job_script.py",
    entry_point="job_script:main",
    wait_until_complete=False,
    job_name="qiskit-braket-vqe-6",
    image_uri=retrieve_image(Framework.BASE, AwsSession().region),
)

In [ ]:
pprint(job.result())

{'VQE': {'eigenstate': {'01': 0.8366600265340756, '11': 0.5477225575051661},
         'eigenvalue': -1.6823050629682066,
         'optimal_parameters': [2.965910707685307,
                                -4.658133055892289,
                                -4.3956015975227265,
                                -3.682189424847994,
                                0.3499389384822065,
                                -2.55160387237748,
                                2.0897918250881915,
                                1.0061448749802206],
         'optimal_point': [2.965910707685307,
                           -4.658133055892289,
                           -4.3956015975227265,
                           -3.682189424847994,
                           0.3499389384822065,
                           -2.55160387237748,
                           2.0897918250881915,
                           1.0061448749802206],
         'optimal_value': -1.6823050629682066}}
